In [3]:
import pandas as pd
import json

In [2]:
df = pd.read_csv('core_db_complete.csv')

In [6]:
def build_llm_input(symptoms, ecg_summary):
    return f"""PATIENT SYMPTOMS:
{symptoms}

ECG SUMMARY:
{ecg_summary}

TASK:
Assess cardiac risk and assign triage level.
List red flags.
Suggest next clinical actions.
Do not provide a diagnosis.
"""



In [7]:
def export_jsonl_for_lora(core_df, out_path="triage_dataset.jsonl"):
    rows = []

    for _, row in core_df.iterrows():
        if not row["ecg_summary"] or not row["symptoms"]:
            continue

        input_text = build_llm_input(
            row["symptoms"],
            row["ecg_summary"]
        )

        record = {
            "input": input_text,
            "output": row["triage_target_json"]
        }

        rows.append(record)

    with open(out_path, "w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

    print(f"[DONE] Exported {len(rows)} examples to {out_path}")
    return rows


In [9]:
dataset_rows = export_jsonl_for_lora(
    df,
    out_path="triage_dataset.jsonl"
)

[DONE] Exported 200 examples to triage_dataset.jsonl
